In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy

# Import linear regression from scipy
from scipy.stats import linregress

# Import requests library
import requests

# Import the weather api key
from config import weather_api_key

# Import datetime
from datetime import datetime

In [2]:
# Generate a set of 1500 random latitudes and longitudes
lats = np.random.uniform(low = -90.000, high = 90.000, size = 1500)
lngs = np.random.uniform(low = -180.000, high = 180.000, size = 1500)

# Zip the lats and lngs together
lat_lngs = zip(lats, lngs)

# Run the lat_lngs
lat_lngs

In [3]:
# Add the lat and lng to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for hold the cities
cities = []

# Identify the nearest city for each lat and lng combo
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    if city not in cities:
        cities.append(city)
        
# Print the city count
print(len(cities))

598


In [5]:
# Build the URL for weather map api call
url = 'https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=' + weather_api_key


In [6]:
# Create an empty list to hold the weather data
city_data = []

# Print the beginning of the logging
print('Beginning Data Retrieval     ')
print('-----------------------------')

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        
    city_url = url + '&q=' + city
    
    # Log the URL, record, and set numbers and the city
    print(f'Processing Record {record_count} of Set {set_count} | {city}')
    
    # Add 1 to the record count
    record_count += 1
    
    # Run an API request for each of the cities
    try:
        
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()
        
        # Parse out the needed data
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity = city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_weather_description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard
        city_date = datetime.utcfromtimestamp(city_weather['dt']).strftime('%Y-%m-%d %H:%M:%S')
        
        # Use try-except blocks to get rain and snowfall in the last 3 hours in inches
        try: 
            
            # Check is there has been rainfall in that last 3 hours, if so, conver to inches
            rain_3h = city_weather['rain']['3h']
            rain_3h_inches = rain_3h / 25.4
            
        except:
            
            # If not rainfall, add 0 to rainfall amount
            rain_3h_inches = 0
            
        try:
            
            # Check is there has been snowfall in that last 3 hours, if so, conver to inches
            snow_3h = city_weather['snow']['3h']
            snow_3h_inches = snow_3h / 25.4
            
        except:
            
            # If not snowfall, add 0 to snowfall amount
            snow_3h_inches = 0
        
        # Append the city information into city_data list
        city_data.append({'City': city.title(),
                         'Country': city_country,
                         'Date': city_date,
                         'Lat': city_lat,
                         'Lng': city_lng,
                         'Max Temp': city_max_temp,
                         'Humidity': city_humidity,
                         'Cloudiness': city_clouds,
                         'Wind Speed': city_wind,
                         'Current Weather': city_weather_description,
                         'Rain Inches (last 3 hrs)': rain_3h_inches,
                         'Snow Inches (last 3 hrs)': snow_3h_inches})
        
    # If an error is experienced, skip the city
    except:
        print('City not found. Skipping...')
        
# Indicated that Data Loading is complete
print('-----------------------------')
print('Data Retrieval Complete      ')
print('-----------------------------')

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | bathsheba
Processing Record 4 of Set 1 | punta arenas
Processing Record 5 of Set 1 | albany
Processing Record 6 of Set 1 | cabo san lucas
Processing Record 7 of Set 1 | mys shmidta
City not found. Skipping...
Processing Record 8 of Set 1 | basco
Processing Record 9 of Set 1 | saint-leu
Processing Record 10 of Set 1 | busselton
Processing Record 11 of Set 1 | kununurra
Processing Record 12 of Set 1 | bacum
Processing Record 13 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 14 of Set 1 | grindavik
Processing Record 15 of Set 1 | vaini
Processing Record 16 of Set 1 | fairbanks
Processing Record 17 of Set 1 | pangnirtung
Processing Record 18 of Set 1 | rungata
City not found. Skipping...
Processing Record 19 of Set 1 | east london
Processing Record 20 of Set 1 | solnechnyy
Processing Record 21 of Set

In [7]:
# Print the length for city_data
print(len(city_data))

545


In [8]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Ushuaia,AR,2020-04-04 19:26:32,-54.80,-68.30,46.40,86,75,13.87,broken clouds,0.00000,0.0
1,Rikitea,PF,2020-04-04 19:26:33,-23.12,-134.97,76.55,79,26,15.84,light rain,0.00748,0.0
2,Bathsheba,BB,2020-04-04 19:26:33,13.22,-59.52,84.20,66,20,11.41,few clouds,0.00000,0.0
3,Punta Arenas,CL,2020-04-04 19:26:34,-53.15,-70.92,44.60,81,75,28.86,light intensity shower rain,0.00000,0.0
4,Albany,US,2020-04-04 19:26:35,42.60,-73.97,61.00,48,75,3.36,broken clouds,0.00000,0.0


In [9]:
# Save the DataFrame into a csv
output_data = 'weather_data/WeatherPy_Challenge.csv'
city_data_df.to_csv(output_data, index_label = 'City_ID')

## How many cities have recorded rainfall or snow? 

In [10]:
rain_snow = city_data_df.loc[(city_data_df['Rain Inches (last 3 hrs)'] >0.0) | (city_data_df['Snow Inches (last 3 hrs)'] >0.0)]

rain_snow.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
1,Rikitea,PF,2020-04-04 19:26:33,-23.12,-134.97,76.55,79,26,15.84,light rain,0.007480,0.0
6,Basco,PH,2020-04-04 19:26:36,20.45,121.97,75.99,84,35,23.42,light rain,0.020079,0.0
7,Saint-Leu,RE,2020-04-04 19:26:37,-21.15,55.28,77.00,78,36,11.41,light rain,0.010236,0.0
24,Butaritari,KI,2020-04-04 19:26:48,3.07,172.79,82.60,80,65,9.57,light rain,0.025984,0.0
45,Manakara,MG,2020-04-04 19:27:01,-22.13,48.02,72.45,86,91,7.96,light rain,0.040551,0.0


In [12]:
rain_snow.count()

City                        99
Country                     99
Date                        99
Lat                         99
Lng                         99
Max Temp                    99
Humidity                    99
Cloudiness                  99
Wind Speed                  99
Current Weather             99
Rain Inches (last 3 hrs)    99
Snow Inches (last 3 hrs)    99
dtype: int64

## Answer: 99 cities have recorded rainfall or snow.